# Twelve Men on an Island: Recursive Programming

    """
    Here we practice OOP to solve the 'Twelve Men on an Island Riddle' (see textfile).
    
    A riddle is represented by:
    1. The number of men of unknown, possibly heavy, possibly light, and normal weight.
    2. The remaining number of times that the scale may be used.
    
    Utilizing the scale decomposes the problem into a number of subproblems described by the same statespace.
    Thus, we will store all subproblems in memory, and break them down further when we want to.
    
    The task is:
    1. For a given problem, generate all unique available moves
    2. Compute the possible outcomes of these moves on the scale (balance or unbalance)
    3. Determine how the outcomes change the state of the problem. Put all unique new states (children) in memory,
        and record their parents.
    4.a See if you have found the solution. If yes, mark the parent as solved
        (and in how many tries this was done)
    4.b See if you have exhausted all children and run out of tries. If yes, mark the parent infeasible
    5. Take your favourite state from memory, and repeat from step 1.
    6. The problem is solved if there is a path where all children are solved (regardless of the scenario)
    
    To make the algorithm smarter, we may want to compute how much information is contained in the statespace,
    and compare it to how many tries are remaining. E.g.
    
    information = (heavy+light+normal)/total + remaining_tries (or something like that)
    
    You can also use states that have been solved earlier (if you record how long it took to solve them)
    
    """

In [ ]:
import itertools
import numpy as np
import time
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
class Riddle:

    # ------------- INIT + OVERRIDE --------------
    def __init__(self, unknown, heavy, light, normal, remaining_tries, start_tries, parent=None, from_move=None, with_tip=None):
        
        self.U = unknown
        self.H = heavy
        self.L = light
        self.N = normal
        self.remaining = remaining_tries
        self.start_tries = start_tries
        self.parent = parent
        self.from_move = from_move
        self.with_tip = with_tip
        
        if ((self.H == 1) and (self.U == 0) and (self.L == 0)) or \
           ((self.L == 1) and (self.U == 0) and (self.H == 0)):
            self.solved =  True
        else:
            self.solved = False
                
        self.total_men = unknown + heavy + light + normal
    
    def __eq__(self, other):
        """Enable equality checks by overriding default behavior. 
        For equality, we don't care about start_tries"""
                
        if isinstance(other, Riddle):
            return ((self.U, self.H, self.L, self.N, self.remaining) == 
                    (other.U, other.H, other.L, other.N, other.remaining))
        
        else:
            return False
    
    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        '''Overwrite default hash'''
        return hash((self.U, self.H, self.L, self.N, self.remaining))
    
    def __str__(self):
        try:        
            return ''' 
            U: {}
            H: {}
            L: {}
            N: {}

            {} tries remaining

            parent = ({},{},{},{})
            from move {}
            with tip {}
            '''.format(self.U, self.H, self.L, self.N, self.remaining,
                      self.parent.U, self.parent.H, self.parent.L, self.parent.N,
                      self.from_move, self.with_tip)
        except:
            return ''' 
            U: {}
            H: {}
            L: {}
            N: {}

            {} tries remaining
            '''.format(self.U, self.H, self.L, self.N, self.remaining)
    
    # ------------- Computing group that is not on the scale --------------
    def notscale_group(self, move):
        nump_people = np.array([self.U,self.H,self.L,self.N]) 

        on_scale = move.sum(axis=0)
        off_scale = nump_people-on_scale
        return off_scale
    
    # ------------- Auxiliary functions for children(): --------------
    
    # GENERATING MOVES: SELF -> MOVES
    def all_moves_generator(self): #self
        u = np.arange(self.U+1)
        h = np.arange(self.H+1)
        l = np.arange(self.L+1)
        n = np.arange(self.N+1)

        um,hm,lm,zm = np.meshgrid(u,h,l,n)

        one_side = np.array([um.flatten(),hm.flatten(),
                             lm.flatten(), zm.flatten()]).T

        # Also only 1/2 vs 1/2 as maximum
        one_side_f = one_side[one_side.sum(axis=1)<=int(0.5*(self.U+self.H+self.L+self.N)),:]

        return np.array(list(itertools.combinations_with_replacement(one_side_f, 2)))
    
    @property
    def moves(self):
        all_moves_np = self.all_moves_generator()
        nump_people = np.array([self.U,self.H,self.L,self.N]) 
        
        nonzero = all_moves_np.sum(axis=1).sum(axis=1)>0
        equal_lr = np.sum(all_moves_np[:,0,:], axis=1) == np.sum(all_moves_np[:,1,:], axis=1)
        available = np.sum(all_moves_np[:,0,:]+all_moves_np[:,1,:] > nump_people, axis=1) == 0

        exc_n_ok = (all_moves_np[:,0,3] == 0) | (all_moves_np[:,1,3] == 0)

        only_h_left = ((all_moves_np[:,0,0] == 0)
                       & (all_moves_np[:,0,2] == 0)
                       & (all_moves_np[:,0,3] == 0)) # only heavy left

        only_h_right = ((all_moves_np[:,1,0] == 0)
                        & (all_moves_np[:,1,2] == 0)
                        & (all_moves_np[:,1,3] == 0)) # only heavy right

        only_l_left = ((all_moves_np[:,0,0] == 0)
                       & (all_moves_np[:,0,1] == 0)
                       & (all_moves_np[:,0,3] == 0)) # only light left

        only_l_right = ((all_moves_np[:,1,0] == 0)
                        & (all_moves_np[:,1,1] == 0)
                        & (all_moves_np[:,1,3] == 0)) # only light right


        only_h_vs_l = ((only_h_left & only_l_right) | (only_h_right & only_l_left)) #?

        return all_moves_np[(nonzero & equal_lr & available & exc_n_ok),:,:] #& ~only_h_vs_l
    

In [ ]:
class Move:
    
    def __init__(self, move, riddle_parent):
        self.move = move
        self.riddle_parent = riddle_parent
        self.unsure = True
        self.solved = False
        self.unsolvable = False
        
    def __eq__(self, other):
        """Enable equality checks by overriding default behavior. 
        For equality, we don't care about start_tries"""
                
        if isinstance(other, Move):
            return ((self.move == other.move).all()) and self.riddle_parent == other.riddle_parent
        
        else:
            return False
    
    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        '''Overwrite default hash'''
        return hash((str(self.move), self.riddle_parent))
    
    def __str__(self):
        return ''' 
        move: {}
        parent: ({}, {}, {}, {}); {} tries remaining
        '''.format(self.move, self.riddle_parent.U, self.riddle_parent.H,
                   self.riddle_parent.L, self.riddle_parent.N,
                   self.riddle_parent.remaining)

        
    # ------------- Creating Riddle Children  --------------------
    @property
    def riddle_children(self):       
        scens = self.scenarios(self.move)
        
        if len(scens) <= 1:
            self.unsolvable = True
            self.unsure = False
            return None

        else:
            move_and_tips = (self.move, scens)
            riddle_children = self.children_per_move(move_and_tips)
            return riddle_children

    # GENERATING OUTCOMES: SELF, MOVE -> (WHICH SIDE CAN GO DOWN)
    def scenarios(self, move):
        scenarios = []

        if (move[:,0].sum()>0) or (move[0,1]>0) or (move[1,2]>0):
            scenarios.append('Left')

        if (move[:,0].sum()>0) or (move[0,2]>0) or (move[1,1]>0):
            scenarios.append('Right')
        
        if sum(move[0,:]+move[1,:]) < self.riddle_parent.total_men: #not all on scale
            rest = self.riddle_parent.notscale_group(move)

            if rest[3] != sum(rest): # if rest is not all normals
                scenarios.append('Balance')
        
        return scenarios
    
    # GENERATING OUTCOMES -> CORRECT?
    def children_per_move(self, move_and_tips):
        children = []
        move = move_and_tips[0]
        tips = move_and_tips[1]
        
        for tip in tips:
            if tip=='Left':
                ###  -------  scale tips left  

                if ~(move[0,3].sum() == move[0,:].sum()) or (move[1,3].sum() == move[1,:].sum()):                
                    U_new = self.riddle_parent.U - move[0,0] - move[1,0] # U_left become H, U_right become L
                    H_new = self.riddle_parent.H + move[0,0] - move[1,1] # U_left become H, H_right become N
                    L_new = self.riddle_parent.L - move[0,2] + move[1,0] # L_left become N, U_right become L
                    N_new = self.riddle_parent.N + move[0,2] + move[1,1] # L_left become N, H_right become N

                    child = Riddle(U_new, H_new, L_new, N_new, self.riddle_parent.remaining-1, self.riddle_parent.start_tries)

                    children.append(child)

                else: # special move: weighing against only normals

                    if move[1,3]>0: #tip left, normal right
                        H_new = move[0,1]+move[0,0]
                        child = Riddle(0, H_new, 0, self.riddle_parent.total_men-H_new, self.riddle_parent.remaining-1, self.riddle_parent.start_tries)

                    elif move[0,3]>0: #tip left, normal left
                        L_new = move[1,2]+move[1,0]
                        child = Riddle(0, 0, L_new, self.riddle_parent.total_men-L_new, self.riddle_parent.remaining-1, self.riddle_parent.start_tries)

                    else:
                        print("ERROR IN SPECIAL CHIILDREN")

                    children.append(child)


            elif tip=='Right':

                if ~(move[0,3].sum() == move[0,:].sum()) or (move[1,3].sum() == move[1,:].sum()):   
                    U_new = self.U - move[0,0] - move[1,0] # U_left become L, U_right become H
                    H_new = self.H - move[0,1] + move[1,0] # H_left becomes N, U_right become H
                    L_new = self.L + move[0,0] - move[1,2] # U_left become L, L_right become N
                    N_new = self.N + move[0,1] + move[1,2] # H_left becomes N, L_right become N

                    child = Riddle(U_new, H_new, L_new, N_new, self.riddle_parent.remaining-1, self.riddle_parent.start_tries, self.riddle_parent.start_tries)

                    children.append(child)

                else: # special move: weighing against only normals

                    if move[1,3]>0: #tip right, normal right
                        L_new = move[0,2]+move[0,0]
                        child = Riddle(0, 0, L_new, self.riddle_parent.total_men-L_new, self.riddle_parent.remaining-1, self.riddle_parent.start_tries)

                    elif move[0,3]>0: #tip right, normal left
                        H_new = move[1,1]+move[1,0]
                        child = Riddle(0, H_new, 0, self.riddle_parent.total_men-H_new, self.riddle_parent.remaining-1, self.riddle_parent.start_tries)

                    else:
                        print("ERROR IN SPECIAL CHIILDREN")

                    children.append(child)

            else: # tip=='Balance'

                ## -- all guys on the scale will go from U,H,L -> N
                U_new = self.riddle_parent.U - move[:,0].sum()
                H_new = self.riddle_parent.H - move[:,1].sum()
                L_new = self.riddle_parent.L - move[:,2].sum()
                N_new = self.riddle_parent.N + move[:,0:3].sum()

                child = Riddle(U_new, H_new, L_new, N_new, self.riddle_parent.remaining-1, self.riddle_parent.start_tries)

                children.append(child)

        return children
        

In [ ]:
# ------------- Unit Tests --------------
def num_people_remains_same(riddle, child):
    return riddle1.total_men == child.total_men

def parent_not_child(riddle, child):
    return ~(riddle == child)


# Main Function

we need to find a path where all children are successful

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(["a", "b", "c"])
G.add_edges_from([("a", "b"), ("a", "c")])
[n for n in G["a"]]
G.add_node(Riddle(1,2,3,4,5,6))
type(Riddle(1,2,3,4,5,6)) == Riddle

In [ ]:
riddle_start = Riddle(3, 0, 0, 1, 3, 3)
visited = []
solved = []
unsure = [riddle_start]
todo = [riddle_start]
unsolvable = []

print("-----------------------------")
print("START")
print("-----------------------------")
print("visited: {}".format(visited))
print("unsure: {}".format(unsure))
print("solved: {}".format(solved))
print("unsolvable: {}".format(unsolvable))
print("todo: {}".format(todo))

G = nx.DiGraph()
G.add_node(riddle_start)

try:
    n = todo.pop(-1)

# If no parent, take a random node in unsure list
except IndexError:
    node_bag = [n for n in unsure if n not in visited]
    n = np.random.choice(node_bag)

if isinstance(n, Riddle):
    if (n not in visited) and (n not in solved):
        if n.remaining >= 1:
            baby_moves = [Move(m, n) for m in n.moves]
            G.add_nodes_from(baby_moves)
            G.add_edges_from([(n, b) for b in baby_moves], direction=-1)
            G.add_edges_from([(b, n) for b in baby_moves], direction=1)
            visited.append(n)
            unsure.append(baby_moves)
    
    if n in visited:
        baby_moves = [k for k, v in G[riddle_start].items() if v["direction"] == -1]
        if any([b.solved for b in baby_moves]):
            n.solved = True
            solved.append(n)
            unsure.remove(n)
        if all([b.unsolvable for b in baby_moves]):
            n.unsolvable = True
            unsolvable.append(n)
            unsure.remove(n)

elif isinstance(n, Move):
    if n not in visited:
        baby_riddles = n.riddle_children
        G.add_nodes_from(baby_children)
        G.add_edges_from([(n, b) for b in baby_children], direction=-1)
        G.add_edges_from([(b, n) for b in baby_children], direction=1)
        visited.append(n)
        
        if all([b.solved]):
            n.solved = True
            solved.append(n)
            unsure.remove(n)
        
    else:
        

print("-----------------------------")
print("ITERATION 1")
print("-----------------------------")
print("visited: {}".format(visited))
print("unsure: {}".format(unsure))
print("solved: {}".format(solved))
print("unsolvable: {}".format(unsolvable))
print("todo: {}".format(todo))

plt.figure(figsize=(36,18))
nx.draw_networkx(G, with_labels=True)

In [ ]:
# DFS
# Start with:
# unsure = [original riddle]
# unsolvable = []  -> list of moves/riddles that for sure will not work
# solved = []
# visited = []

# Try to access parent(s) of previous node (careful with riddles, they may have multiple move parents!)
# If no parent, take a random node in unsure list
# For that node:

# IF IT IS A RIDDLE             
# Check if it has already been visited
    # if not visited and not solved:
        # if there is > 1 try remaining
            # Generate all moves (set unsolved), add to unsolved list
            # Add them as nodes
            # Add edges between the moves and the previous riddle
            # mark riddle as visited
        # else (run out of moves):
            # mark as wrong (remove from unsolved)
    # if visited:
        # check if there is at least one baby move that is solved
            # if yes, mark riddle as solved (remove from unsolved)
        # check if there all baby moves are wrong, if yes, mark riddle wrong
    # check if the original riddle is solved
            
# IF IT IS A MOVE
    # if not visited:
        # get riddle parent
        # generate all baby children
        # check if all baby children are solved immediately, if so, mark move as solved
    # if visited:
        # check if all baby moves are solved, if so, mark move as solved
        # check if there is one or more baby riddles that are wrong, if so, mark wrong


In [ ]:
G = nx.Graph()

In [ ]:
G.add_node(r, kind="riddle")

In [ ]:
move_nodes = [str((r, i)) for i in r.make_feasible_moves_np()]
move_nodes

In [ ]:
new_edges = [(r, mn) for mn in move_nodes]
new_edges

In [ ]:
G.add_nodes_from(move_nodes, kind="move")

In [ ]:
G.add_edges_from(new_edges)

In [ ]:
nx.draw(G)

In [ ]:
r = Riddle(4, 0, 0, 0, 3, 3)
print(r.make_feasible_moves_np())

In [ ]:
# Pick a node
# if kind == riddle
    # generate all moves (mark all moves as unsolved)
    # add moves as nodes
    # add edges between riddle and moves
    
# elif kind == move
    # generate all children riddles
    # check if all of them are solved
        # if yes, mark the move solved
    

# Hashing
https://www.asmeurer.com/blog/posts/what-happens-when-you-mess-with-hashing-in-python/

One of the key points that I hope you will take away from this post is that if you override __eq__ and you want a hashable object, you must also override __hash__ to agree.

The hash of an object does not change across the object's lifetime (in other words, a hashable object should be immutable).

a == b implies hash(a) == hash(b) (note that the reverse might not hold in the case of a hash collision).

# Caching
https://www.thepythoncorner.com/2018/04/how-to-make-your-code-faster-by-using-a-cache-in-python/?source=post_page---------------------------&doing_wp_cron=1564991913.5016050338745117187500

https://www.youtube.com/watch?v=EsUTO4Xzehg

# Moves

represented as a tuple of tuples:

move = ((U, H, L, N), (U, H, L, N))

constraints (no stupid moves):

- sum(move[0]) == sum(move[1]) (equal number of people left and right of the scale)
- can only put people on the scale that are available in that state
- moves should be reduced. Determine equivalent moves:
    - adding normals on both sides does not help.
        e.g. ((2,0,1,5), (1,2,2,3)) == ((2,0,1,2), (1,2,2,0)) (deleting three normals)
    - never do only heavy vs only light
    - ...
    
Went over to numpy, much faster in filtering feasible from non-feasible moves.

Does not make much difference in generating all possible moves?

Used itertools combinations instead of product to remove symmetric moves

NEEDS VALIDATION, BUT DONE

In [ ]:
# Time decorator
def time_func(func):
    def wrapper(*args, **kwargs):
        
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print('executing {} took {} seconds'.format(func.__name__, str(end-start)))
        
        return result
    return wrapper

# Human thought process

In [ ]:
state = Riddle(None,None,None,0,3,0,1,3,3) #None parent, None from_move, None with_tip
print(state)

In [ ]:
# get all movesb I can do
moves = state.make_feasible_moves_np()
moves

In [ ]:
# Take 1 move
take = 0
move = moves[take]
move

In [ ]:
# Get the outcomes of that move
state.children_listed[take]

In [ ]:
# Check if they are solved
print('first child solved:' + str(state.children_listed[take][0].solved_self))
print('second child solved:' + str(state.children_listed[take][1].solved_self))
# print('third child solved:' + str(state.children_listed[take][2].solved_self))


In [ ]:
# take a child that is not solved
state = state.children_listed[take][0]

# generate all moves

# take 1 move

# Generate the outcomes of that move


In [ ]:
state = Riddle(None,None,None,2,0,0,1,3,3) #None parent, None from_move, None with_tip
state.children_listed

stack = [state]

# while stack:
if all([s.solved_self for s in stack]):
    print(True)
else:
    stack.pop(0)
    stack = stack+state.children_listed

print(stack)

# state.solved

# # def solve(state):
# if state.solved_self:
#     state.solved = True
# else:
#     state.solved = any([all([c.solved_self for c in c_s]) for c_s in state.children_listed])

# if not state.solved:
#     state.solved = solve(state)
    
    
#     for c_s in state.children_listed:
#         print(c_s)
#         print(all([c.solved for c in c_s]))
        
        
#         print('=====')
#         print(c_s)
#     all([c.solved for c in ])



# initial_state.children_listed
    

In [ ]:
r = Riddle(None,None,None,2,0,0,1,3,2) #None parent, None from_move, None with_tip
all_moves = r.make_feasible_moves_np()
all_scenarios = [r.scenarios(move) for move in all_moves]
fil = list(map(lambda x: len(x)>1, all_scenarios))

all_moves = list(itertools.compress(all_moves, fil))
all_scenarios = list(itertools.compress(all_scenarios, fil))
moves_and_tips = list(zip(all_moves, all_scenarios))
all_moves

In [ ]:
print(r)

In [ ]:
r.solved

In [ ]:
for c in r.children:
    print(c.solved_self)

In [ ]:
c = r.children[3]
print(c)
print('c is solved:' + str(c.solved_self))

In [ ]:
for cc in c.children:
    print(cc.solved_self)

In [ ]:
for c in r.children:
    print(c.solved_self)

In [ ]:
for c_d in r.children:
    print('=============')
    print('''By executing move 
    {} 
    
    for ({},{},{},{},{})
    
    You can end up in'''.format(c_d.from_move, c_d.parent.U, c_d.parent.H, 
                                c_d.parent.L, c_d.parent.N, c_d.parent.remaining))
    
    print(c_d)
    
    print('By tipping {}'.format(c_d.with_tip))

In [ ]:
r2 = r.children[-1]
print(r2)

In [ ]:
r2.solved_self

In [ ]:
print(c_d.solved_self)

In [ ]:
for c_d in r2.children:
    print('=============')
    print('''By executing move 
    {} 
    
    for ({},{},{},{},{})
    
    You can end up in'''.format(c_d.from_move, c_d.parent.U, c_d.parent.H, 
                                c_d.parent.L, c_d.parent.N, c_d.parent.remaining))
    
    print(c_d)
    
    print('By tipping {}'.format(c_d.with_tip))

# Experimentation

In [ ]:
# checking hash of same problem but different start_tries
riddle1 = Riddle(12,0,0,0,3,3,False)
riddle2 = Riddle(12,0,0,0,5,3,False)
riddle3 = Riddle(12,0,0,0,6,3,False)

print('hash same:' + str(hash(riddle1) == hash(riddle2)))
print(' == same:' + str(riddle1 == riddle2))

In [ ]:
# behaviour in list
l = [riddle1, riddle2]
list(map(lambda x: x==riddle3, l))

In [ ]:
# behaviour of overwriting
riddle1.U=5
print(riddle1)
print(riddle2)